<a href="https://colab.research.google.com/github/hakui-note/Numer0n/blob/main/Numer0n.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## プロトタイプ

In [ ]:
import random

def create_answer():
    """
    コンピュータの答えとなる3桁のユニークな数字を文字列として生成します。
    例: "012", "753"
    """
    # 0から9までの数字のリストから、重複しないように3つをランダムに選びます。
    digits = "0123456789"
    answer = "".join(random.sample(digits, 3))
    return answer

def check_guess(guess, answer):
    """
    プレイヤーの予想を判定し、EATとBITEの数をタプルで返します。
    (eat, bite)
    """
    eat = 0
    bite = 0

    # EATの数を数えます (数字と位置が両方合っている)
    for i in range(len(answer)):
        if guess[i] == answer[i]:
            eat += 1

    # BITEの数を数えます (数字は合っているが位置が違う)
    # 1. 答えと予想に含まれる共通の数字の数を数えます。
    #    (setの積集合を使うと簡単に計算できます)
    common_digits_count = len(set(guess) & set(answer))

    # 2. 共通の数字の数から、位置も合っているEATの数を引くことで、
    #    BITEの数が求められます。
    bite = common_digits_count - eat

    return eat, bite

def main():
    """
    ゲームのメイン処理を行う関数です。
    """
    # ゲームの初期設定
    answer = create_answer()
    turn = 0

    # ゲーム開始のメッセージ
    print("--- ヌメロンゲーム スタート！ ---")
    print("私が考えた3桁の数字を当ててください。")
    print("ルール:")
    print("・3桁の数字です。")
    print("・同じ数字は使われていません。")
    print("・先頭が0の場合もあります (例: 012)。")
    print("---------------------------------")

    # デバッグ用に答えを確認したい場合は、以下の行のコメントを解除してください
    # print(f"（デバッグ用）答え: {answer}")

    # メインループ (3 EATになるまで続く)
    while True:
        turn += 1

        # プレイヤーからの入力を受け付け、入力が正しいかチェックします。
        while True:
            guess = input(f"\n【{turn}回目】3桁の数字を予想してください: ")

            # 入力値のバリデーション
            if len(guess) != 3:
                print("エラー: 3桁の数字を入力してください。")
            elif not guess.isdigit():
                print("エラー: 半角数字で入力してください。")
            elif len(set(guess)) != 3:
                print("エラー: 重複しない3つの数字を入力してください。")
            else:
                # 正しい入力であればチェックを終了
                break

        # EATとBITEを判定
        eat, bite = check_guess(guess, answer)

        # 正解の場合
        if eat == 3:
            print(f"\n結果: 3 EAT, 0 BITE")
            print("=========================================")
            print(f"🎉 おめでとうございます！見事正解です！ 🎉")
            print(f"正解は「{answer}」でした。")
            print(f"{turn}回でクリアしました。")
            print("=========================================")
            break
        # 不正解の場合
        else:
            print(f"結果: {eat} EAT, {bite} BITE")

# このスクリプトが直接実行された場合にmain()関数を呼び出す
if __name__ == "__main__":
    main()

--- ヌメロンゲーム スタート！ ---
私が考えた3桁の数字を当ててください。
ルール:
・3桁の数字です。
・同じ数字は使われていません。
・先頭が0の場合もあります (例: 012)。
---------------------------------

【1回目】3桁の数字を予想してください: 123
結果: 0 EAT, 2 BITE

【2回目】3桁の数字を予想してください: 412
結果: 1 EAT, 0 BITE

【3回目】3桁の数字を予想してください: 516
結果: 1 EAT, 0 BITE

【4回目】3桁の数字を予想してください: 713
結果: 0 EAT, 1 BITE

【5回目】3桁の数字を予想してください: 398
結果: 0 EAT, 1 BITE

【6回目】3桁の数字を予想してください: 235
結果: 1 EAT, 2 BITE

【7回目】3桁の数字を予想してください: 532

結果: 3 EAT, 0 BITE
🎉 おめでとうございます！見事正解です！ 🎉
正解は「532」でした。
7回でクリアしました。


## タイマーを追加

In [ ]:
import random
import time
import threading
import sys

# --- ゲームの状態を管理する共有オブジェクト ---
# スレッド間で情報を共有するために辞書を使用します。
game_state = {
    'time_left': 10 * 60,  # 制限時間: 10分 = 600秒
    'game_over': False     # ゲーム終了フラグ (時間切れ or 正解)
}

def create_answer():
    """コンピュータの答えとなる3桁のユニークな数字を生成します。"""
    digits = "0123456789"
    answer = "".join(random.sample(digits, 3))
    return answer

def check_guess(guess, answer):
    """プレイヤーの予想を判定し、EATとBITEの数を返します。"""
    eat = 0
    # EATの数を数える (数字も位置も合っている)
    for i in range(len(answer)):
        if guess[i] == answer[i]:
            eat += 1

    # BITEの数を数える (数字は合っているが位置が違う)
    common_digits_count = len(set(guess) & set(answer))
    bite = common_digits_count - eat

    return eat, bite

def countdown_timer():
    """
    バックグラウンドで実行されるカウントダウンタイマー関数。
    1秒ごとに残り時間を更新します。
    """
    # game_overフラグが立つか、時間がなくなるまでループ
    while game_state['time_left'] > 0 and not game_state['game_over']:
        # 1秒待機
        time.sleep(1)
        # 残り時間を1秒減らす
        game_state['time_left'] -= 1

    # 時間切れになった場合（game_overフラグがまだFalseの場合）
    if not game_state['game_over']:
        # タイムアップを通知し、ゲーム終了フラグを立てる
        print("\n\n*** 時間切れです！ ゲームオーバー ***")
        game_state['game_over'] = True

def main():
    """ゲームのメイン処理を行う関数です。"""
    answer = create_answer()
    turn = 0

    # --- ゲーム開始のメッセージ ---
    print("--- ヌメロンゲーム スタート！ ---")
    print("私が考えた3桁の数字を当ててください。")
    print("🔥 制限時間は10分です！ 🔥")

    # --- タイマースレッドを開始 ---
    # countdown_timer関数を別のスレッドで実行します。
    # daemon=Trueにすると、メインプログラムが終了したときにこのスレッドも自動的に終了します。
    timer_thread = threading.Thread(target=countdown_timer, daemon=True)
    timer_thread.start()

    # --- メインループ (ゲームオーバーになるまで続く) ---
    while not game_state['game_over']:
        turn += 1

        # 現在の残り時間を表示
        mins, secs = divmod(game_state['time_left'], 60)
        print("---------------------------------")
        print(f"残り時間: {mins:02d}分{secs:02d}秒")

        # プレイヤーからの入力を受け付けます
        try:
            guess = input(f"【{turn}回目】3桁の数字を予想してください: ")
        except (EOFError, KeyboardInterrupt):
            # Ctrl+Cなどで強制終了した場合
            game_state['game_over'] = True
            print("\nゲームを中断しました。")
            continue # ループを抜けるためにcontinue

        # 入力後に時間切れになっていないか再度チェック
        if game_state['game_over']:
            print("入力しましたが、時間切れでした。")
            break

        # 入力値のバリデーション
        if len(guess) != 3:
            print("エラー: 3桁の数字を入力してください。")
            continue
        elif not guess.isdigit():
            print("エラー: 半角数字で入力してください。")
            continue
        elif len(set(guess)) != 3:
            print("エラー: 重複しない3つの数字を入力してください。")
            continue

        # EATとBITEを判定
        eat, bite = check_guess(guess, answer)

        # 正解の場合
        if eat == 3:
            game_state['game_over'] = True # タイマースレッドを止めるためにフラグを立てる
            print(f"\n結果: 3 EAT, 0 BITE")
            print("=========================================")
            print(f"🎉 おめでとうございます！見事正解です！ 🎉")
            print(f"正解は「{answer}」でした。")
            print(f"{turn}回でクリアしました。")

            # クリア時の残り時間を表示
            mins, secs = divmod(game_state['time_left'], 60)
            print(f"残り時間: {mins:02d}分{secs:02d}秒")
            print("=========================================")
        # 不正解の場合
        else:
            print(f"結果: {eat} EAT, {bite} BITE")

    # ゲーム終了時のメッセージ
    # 時間切れで、かつまだ答えが表示されていない場合
    if game_state['time_left'] <= 0 and eat != 3:
         print(f"正解は「{answer}」でした。")

    print("\nゲームを終了します。")

# このスクリプトが直接実行された場合にmain()関数を呼び出す
if __name__ == "__main__":
    main()

--- ヌメロンゲーム スタート！ ---
私が考えた3桁の数字を当ててください。
🔥 制限時間は10分です！ 🔥
---------------------------------
残り時間: 10分00秒
【1回目】3桁の数字を予想してください: 012
結果: 0 EAT, 2 BITE
---------------------------------
残り時間: 09分54秒
【2回目】3桁の数字を予想してください: 301
結果: 0 EAT, 1 BITE
---------------------------------
残り時間: 09分23秒
【3回目】3桁の数字を予想してください: 241
結果: 0 EAT, 2 BITE
---------------------------------
残り時間: 08分26秒
【4回目】3桁の数字を予想してください: 125
結果: 2 EAT, 0 BITE
---------------------------------
残り時間: 07分49秒
【5回目】3桁の数字を予想してください: 126

結果: 3 EAT, 0 BITE
🎉 おめでとうございます！見事正解です！ 🎉
正解は「126」でした。
5回でクリアしました。
残り時間: 07分27秒

ゲームを終了します。


### さらに難易度を上げる
4桁に増やす

In [ ]:
import random
import time
import threading
import sys

# --- ゲームの状態を管理する共有オブジェクト ---
game_state = {
    'time_left': 10 * 60,  # 制限時間: 10分 = 600秒
    'game_over': False     # ゲーム終了フラグ (時間切れ or 正解)
}

def create_answer():
    """コンピュータの答えとなる4桁のユニークな数字を生成します。"""
    digits = "0123456789"
    # 3桁から4桁に変更
    answer = "".join(random.sample(digits, 4))
    return answer

def check_guess(guess, answer):
    """プレイヤーの予想を判定し、EATとBITEの数を返します。"""
    eat = 0
    # EATの数を数える (数字も位置も合っている)
    for i in range(len(answer)):
        if guess[i] == answer[i]:
            eat += 1

    # BITEの数を数える (数字は合っているが位置が違う)
    common_digits_count = len(set(guess) & set(answer))
    bite = common_digits_count - eat

    return eat, bite

def countdown_timer():
    """
    バックグラウンドで実行されるカウントダウンタイマー関数。
    1秒ごとに残り時間を更新します。
    """
    # game_overフラグが立つか、時間がなくなるまでループ
    while game_state['time_left'] > 0 and not game_state['game_over']:
        # 1秒待機
        time.sleep(1)
        # 残り時間を1秒減らす
        game_state['time_left'] -= 1

    # 時間切れになった場合（game_overフラグがまだFalseの場合）
    if not game_state['game_over']:
        # タイムアップを通知し、ゲーム終了フラグを立てる
        print("\n\n*** 時間切れです！ ゲームオーバー ***")
        game_state['game_over'] = True

def main():
    """ゲームのメイン処理を行う関数です。"""
    answer = create_answer()
    turn = 0

    # --- ゲーム開始のメッセージ ---
    print("--- ヌメロンゲーム スタート！ ---")
    # 3桁から4桁に変更
    print("私が考えた4桁の数字を当ててください。")
    print("🔥 制限時間は10分です！ 🔥")

    # --- タイマースレッドを開始 ---
    timer_thread = threading.Thread(target=countdown_timer, daemon=True)
    timer_thread.start()

    # --- メインループ (ゲームオーバーになるまで続く) ---
    eat = 0 # ループの外でeatを初期化
    while not game_state['game_over']:
        turn += 1

        # 現在の残り時間を表示
        mins, secs = divmod(game_state['time_left'], 60)
        print("---------------------------------")
        print(f"残り時間: {mins:02d}分{secs:02d}秒")

        # プレイヤーからの入力を受け付けます
        try:
            # 3桁から4桁に変更
            guess = input(f"【{turn}回目】4桁の数字を予想してください: ")
        except (EOFError, KeyboardInterrupt):
            game_state['game_over'] = True
            print("\nゲームを中断しました。")
            continue

        # 入力後に時間切れになっていないか再度チェック
        if game_state['game_over']:
            print("入力しましたが、時間切れでした。")
            break

        # 入力値のバリデーション (3桁から4桁に変更)
        if len(guess) != 4:
            print("エラー: 4桁の数字を入力してください。")
            continue
        elif not guess.isdigit():
            print("エラー: 半角数字で入力してください。")
            continue
        # 3桁から4桁に変更
        elif len(set(guess)) != 4:
            print("エラー: 重複しない4つの数字を入力してください。")
            continue

        # EATとBITEを判定
        eat, bite = check_guess(guess, answer)

        # 正解の場合 (3 EATから 4 EAT に変更)
        if eat == 4:
            game_state['game_over'] = True
            # 3 EATから 4 EAT に変更
            print(f"\n結果: 4 EAT, 0 BITE")
            print("=========================================")
            print(f"🎉 おめでとうございます！見事正解です！ 🎉")
            print(f"正解は「{answer}」でした。")
            print(f"{turn}回でクリアしました。")

            mins, secs = divmod(game_state['time_left'], 60)
            print(f"残り時間: {mins:02d}分{secs:02d}秒")
            print("=========================================")
        # 不正解の場合
        else:
            print(f"結果: {eat} EAT, {bite} BITE")

    # ゲーム終了時のメッセージ
    # 時間切れで、かつまだ答えが表示されていない場合 (3 EATから 4 EAT に変更)
    if game_state['time_left'] <= 0 and eat != 4:
         print(f"正解は「{answer}」でした。")

    print("\nゲームを終了します。")

if __name__ == "__main__":
    main()

--- ヌメロンゲーム スタート！ ---
私が考えた4桁の数字を当ててください。
🔥 制限時間は10分です！ 🔥
---------------------------------
残り時間: 10分00秒
【1回目】4桁の数字を予想してください: 0123
結果: 0 EAT, 2 BITE
---------------------------------
残り時間: 09分54秒
【2回目】4桁の数字を予想してください: 4501
結果: 1 EAT, 1 BITE
---------------------------------
残り時間: 09分19秒
【3回目】4桁の数字を予想してください: 6107
結果: 1 EAT, 1 BITE
---------------------------------
残り時間: 08分48秒
【4回目】4桁の数字を予想してください: 1809
結果: 1 EAT, 0 BITE
---------------------------------
残り時間: 08分15秒
【5回目】4桁の数字を予想してください: 2908
結果: 2 EAT, 0 BITE
---------------------------------
残り時間: 06分44秒
【6回目】4桁の数字を予想してください: 2301
結果: 2 EAT, 0 BITE
---------------------------------
残り時間: 06分05秒
【7回目】4桁の数字を予想してください: 2406

結果: 4 EAT, 0 BITE
🎉 おめでとうございます！見事正解です！ 🎉
正解は「2406」でした。
7回でクリアしました。
残り時間: 05分19秒

ゲームを終了します。
